In [6]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import threading
from dash import Dash, dcc, html, Input, Output, jupyter_dash

%matplotlib inline

In [7]:
mapbox_token = "pk.eyJ1IjoidmVydWJlZSIsImEiOiJjbTJxOXR5ZngxMTF1Mmtxenp6aW16eTdjIn0.N-u6aPAAJvjY44ZLWfZPmw"

px.set_mapbox_access_token(mapbox_token)

In [8]:
df = pd.read_csv('Final_projects_files/project_1_python.csv', index_col='index',sep=',', decimal='.', encoding='UTF-8')

df.tail(1)

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,total_tests,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
index,,,,,,,,,,,,,,,,,,,,,
190612,ZWE,Africa,Zimbabwe,2022-06-18,254753.0,NaN,5533.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,15092171.0,19.6,1899.775,61.49,-20.0,30.0


In [10]:
app = Dash(__name__)
app.layout = html.Div([
    html.H1(id='title', children='COVID-19 Cumulative Cases and Deaths', style={ "color": "#3498DB", "text-align": "center"}),
    
    html.H3("Select Country:", style={"text-align": "center", "color": "#AED6F1"}),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df['location'].unique()],
        value='Afghanistan',
        style= {"text-align": "center"}
    ),
    
    html.Div([
        dcc.Graph(id='cumulative-cases-chart'),
        dcc.Graph(id='cumulative-deaths-chart')
    ], style={'display': 'flex', 'flex-direction': 'row'})
])

@app.callback(
    [Output('cumulative-cases-chart', 'figure'),
     Output('cumulative-deaths-chart', 'figure'),
     Output('title', 'children')],
    [Input('country-dropdown', 'value')]
)
def update_charts(selected_country):
    filtered_df = df[df['location'] == selected_country]

    # Line chart - cumulative cases
    fig_cases = px.line(filtered_df, x='date', y='total_cases', 
                        title=f'Cumulative Cases in {selected_country}')
    fig_cases.update_traces(line=dict(color='orange'))
    fig_cases.update_layout(yaxis_title='Cumulative Cases', xaxis_title='Date', width=520,)

    # Line chart - cumulative deaths
    fig_deaths = px.line(filtered_df, x='date', y='total_deaths', 
                         title=f'Cumulative Deaths in {selected_country}')
    fig_deaths.update_traces(line=dict(color='red'))
    fig_deaths.update_layout(yaxis_title='Cumulative Deaths', xaxis_title='Date', width=520,)
    
    title = f'Cumulative COVID-19 Cases and Deaths in {selected_country}'
    return fig_cases, fig_deaths, title

if __name__ == '__main__':
   app.run_server(mode='inline', port=8053)


